
<center> <h1 style="background-color:orange; color:white"><br>Detalles del dataset<br></h1></center>

En el comercio electrónico, recomendar productos relevantes a los clientes es clave para mejorar la experiencia de compra y aumentar las ventas. Uno de los enfoques más utilizados para este propósito es la **recomendación basada en vecinos cercanos**, también conocida como **Collaborative Filtering (Filtrado Colaborativo)**.  

##  ¿Cómo Funciona?  
El método de recomendación por vecinos cercanos se basa en el análisis de patrones de compra de distintos clientes. La idea principal es que los usuarios con comportamientos de compra similares probablemente estén interesados en productos similares.  

### Enfoques Principales  
Existen dos enfoques clave dentro de este método:  

1. **Filtrado basado en usuarios:**  
   - Se identifican clientes con hábitos de compra similares.  
   - Se recomiendan productos que compraron clientes con perfiles similares.  

2. **Filtrado basado en ítems:**  
   - Se analizan productos que suelen comprarse juntos.  
   - Se recomiendan productos que frecuentemente aparecen en el historial de compra de otros clientes con intereses similares.  

## Beneficios de Este Método  
✔️ **Personalización:** Ofrece recomendaciones adaptadas a cada cliente.  
✔️ **Eficiencia:** No requiere información detallada sobre los productos, solo el historial de compras.  
✔️ **Escalabilidad:** Funciona bien con grandes volúmenes de datos en plataformas de e-commerce.  
✔️ **Explicabilidad:** Aporta información facil de entender respecto al funcionamiento de las recomendaciones, por ejemplo, permite comprender cuales son los usuarios cercanos y cuales son los items más influyentes en la recomendación.  

Este método es ampliamente utilizado por empresas como Amazon, Netflix y Spotify para mejorar la relevancia de sus recomendaciones y potenciar la fidelización de clientes.  




<center> <h1 style="background-color:orange; color:white"><br>Detalles del dataset<br></h1></center>

<br>
Un e-commerce busca segmentar a sus clientes y definir estrategias de marketing basadas en estos segmentos. Para ello, analizaremos el comportamiento de los clientes y crearemos grupos utilizando knn como tecnica de cluster. En otras palabras, agruparemos a los clientes con patrones de comportamiento similares y diseñaremos estrategias de ventas y marketing personalizadas para cada grupo.
<br><br>

<p>
- <b>InvoiceNo:</b> Número de factura. Nominal. Es un número entero de 6 dígitos asignado de manera única a cada transacción. Si el código comienza con la letra 'C', indica una cancelación.<br />
- <b>StockCode:</b> Código de producto (item). Nominal. Es un número entero de 5 dígitos asignado de manera única a cada producto.<br />
- <b>Description:</b> Nombre del producto (item). Nominal.<br />
- <b>Quantity:</b> Cantidad de unidades de cada producto por transacción. Numérico.<br />
- <b>InvoiceDate:</b> Fecha y hora de la factura. Numérico. Indica el día y la hora en que se generó una transacción.<br />
- <b>UnitPrice:</b> Precio unitario. Numérico. Representa el precio de cada producto en libras esterlinas (£).<br />
- <b>CustomerID:</b> Número de cliente. Nominal. Es un número entero de 5 dígitos asignado de manera única a cada cliente.<br />
- <b>Country:</b> País. Nominal. Indica el país de residencia del cliente.<br />

<a id='1'></a><center> <h1 style="background-color:orange; color:white" ><br>Analisis exploratorio y limpieza de datos<br></h1>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine

import warnings

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
df = pd.read_csv("online_retail_listing.csv",delimiter=';',encoding="latin-1")

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,"6,95",13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,"2,1",13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,"1,25",13085.00,United Kingdom


In [4]:
df.shape

(1048575, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1048575 non-null  object 
 1   StockCode    1048575 non-null  object 
 2   Description  1044203 non-null  object 
 3   Quantity     1048575 non-null  int64  
 4   InvoiceDate  1048575 non-null  object 
 5   Price        1048575 non-null  object 
 6   Customer ID  811893 non-null   float64
 7   Country      1048575 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 64.0+ MB


In [ ]:
#Transformamos la columna 'InvoiceDate' a datetime y luego, transformamos esta fecha a la cantidad de dias de diferencia respecto a 1970, de manera que en un futuro pueda usarse la fecha con algoritmos de knn
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],format="%d.%m.%Y %H:%M")
df['InvoiceDays'] = df['InvoiceDate'].astype('int64') // int(pd.Timedelta(days=1) / pd.Timedelta('1ns'))
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,InvoiceDays
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,"6,95",13085.00,United Kingdom,14579
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,"6,75",13085.00,United Kingdom,14579
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,"6,75",13085.00,United Kingdom,14579
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,"2,1",13085.00,United Kingdom,14579
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,"1,25",13085.00,United Kingdom,14579


In [7]:
# ¿Cuantos productos distintos se venden?
df["Description"].nunique()

5697

In [ ]:
#¿Cuanto se paga por Invoce?
# Transformo el precio a Float
df["Price"] = df["Price"].apply(lambda x: float(str(x.replace(',','.'))))
df["TotalPrice"] = df["Quantity"]*df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,InvoiceDays,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom,14579,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,14579,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,14579,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom,14579,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom,14579,30.00


In [9]:
df.groupby("Invoice").agg({"TotalPrice":"sum"}).head()

,TotalPrice
Invoice,
489434,505.30
489435,145.80
489436,630.33
489437,310.75
489438,2286.24


In [ ]:
#Chequeo si hay valores nulos
df.isnull().sum()

Invoice             0
StockCode           0
Description      4372
Quantity            0
InvoiceDate         0
Price               0
Customer ID    236682
Country             0
InvoiceDays         0
TotalPrice          0
dtype: int64

In [ ]:
# Borro valores nulos
df.dropna(inplace = True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
InvoiceDays    0
TotalPrice     0
dtype: int64

In [ ]:
#Busco los outliers por cantidad, precio y precio total. 
for feature in ["Quantity","Price","TotalPrice"]:

    Q1 = df[feature].quantile(0.01)
    Q3 = df[feature].quantile(0.99)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature, "no")

Quantity yes
2142
Price yes
1789
TotalPrice yes
2113


<a id='8'></a><center> <h1 style="background-color:orange; color:white" ><br>Constucción del modelo KNN<br></h1>

**Lógica de KNN:**
 KNN (K-Nearest Neighbors) es un algoritmo basado en la similitud entre elementos. En el caso de recomendaciones de productos, se calcula la distancia entre vectores de productos (por ejemplo, en función de sus características como precio, cantidad vendida, categoría, etc.).

**Métrica de distancia:** 
Se pueden usar diferentes métricas como la distancia euclidiana o el coseno de similitud. En general:

<u>Distancia Euclidiana:</u> Ideal cuando los datos están en un espacio continuo y tienen unidades similares.

<u>Similitud del Coseno:</u> Útil para comparar vectores de características cuando importa más la dirección que la magnitud.

Si un cliente objetivo compra ciertos productos que son recurrentes en otros clientes, es probabable que el cliente objetivo quiera comprar otros productos de los clientes con compras similares.

De esta forma, KNN encuentra a los usuarios(vecinos) más cercanos - en el sentido de consumos similares - dentro del espacio vectorial conformado por la cantidad comprada de cada producto.


Proceso:

1. Se selecciona un producto de referencia.

2. Se calculan las distancias entre este producto y todos los demás en la base de datos.

3. Se ordenan los productos según la menor distancia.

4. Se eligen los k productos más cercanos como recomendaciones.

Si un cliente objetivo compra ciertos productos que son recurrentes en otros clientes, es probabable que el cliente objetivo quiera compr
De esta forma, si un cliente objetivo compra ciertos productos que son recurrentes en otros clientes, es probabable que el cliente objetivo quiera compr

In [46]:
#Hacemos un analisis independiente del tiempo, por lo tanto vamos a sumar la cantidad total comprada de cada producto por cada cliente
df_agrupado = df.groupby(['Customer ID', 'Description'], as_index=False)['Quantity','TotalPrice'].sum()

#Acortamos el dataframe para probar el algoritmo
df_prueba= df_agrupado.head(10000)


#Diccionario de usuarios, relacionamos cada Customer ID a un indice
users = df_prueba['Customer ID'].drop_duplicates().tolist()

user_ids_dict = {index: user for user, index in enumerate(users)}

#Distintos Items
items = df_prueba['Description'].drop_duplicates()
items = items.tolist()

u, i =  len(users), len(items) 

# Crear una matriz de ceros de dimension usuarios x productos
matriz = np.zeros((u, i+1))


C:\Users\santi\AppData\Local\Temp\ipykernel_2892\3676109827.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_agrupado = df.groupby(['Customer ID', 'Description'], as_index=False)['Quantity','TotalPrice'].sum()


In [47]:
print(df_agrupado)

        Customer ID                          Description  Quantity  TotalPrice
0          12346.00  Adjustment by john on 26/01/2010 17        -1     -103.50
1          12346.00                DOORMAT 3 SMILEY CATS         1        7.49
2          12346.00                     DOORMAT AIRMAIL          1        7.49
3          12346.00                 DOORMAT BLACK FLOCK          1        7.49
4          12346.00            DOORMAT CHRISTMAS VILLAGE         1        7.49
...             ...                                  ...       ...         ...
490108     18287.00                  TOADSTOOL MONEY BOX         6       17.70
490109     18287.00    TREE T-LIGHT HOLDER WILLIE WINKIE        12       19.80
490110     18287.00               WATERMELON BATH SPONGE        20       25.00
490111     18287.00      WHITE CHRISTMAS STAR DECORATION        36       15.12
490112     18287.00          WOODEN BOX ADVENT CALENDAR          4       35.80

[490113 rows x 4 columns]


In [51]:
#llenamos la matriz con la cantidad de cada producto consumida por cada cliente
for user in users:
    for i in range(len(items)+1):
        user_index = user_ids_dict[user]
        if i == len(items):
            matriz[user_index][i] = df_prueba[(df_prueba['Customer ID'] == user) & (df_prueba['Description'] == item)]['TotalPrice'].sum()
        else:
            item = items[i]
            matriz[user_index][i] = df_prueba[(df_prueba['Customer ID'] == user) & (df_prueba['Description'] == item)]['Quantity'].sum()
        

In [52]:
#Normalizamos
mean_matriz = np.mean(matriz, axis=1).reshape(-1, 1)
matriz_normalized = matriz - mean_matriz

In [53]:
# Aplicamos KNN
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(matriz_normalized)

NearestNeighbors(algorithm='brute', metric='cosine')

In [54]:
#Buscamos los k vecinos cercanos a un usuario particular
def kneighborsForUser(k,user):

    target_user_index = int(user_ids_dict[user])
    distances, indices = knn.kneighbors(matriz_normalized[target_user_index].reshape(1, -1), n_neighbors=k)

    return distances, indices 


In [55]:
#ranking de los vecinos más cercanos
def ratingsFromNeighbors(distances, indices):
    # Aggregate ratings from the nearest neighbors
    neighbors_ratings = matriz[indices.flatten()]
    predicted_ratings = neighbors_ratings.mean(axis=0)
    return predicted_ratings

In [78]:
def encontrar_id_por_indice(diccionario, indice_buscado):
    for id, indice in diccionario.items():
        if indice == indice_buscado:
            return id
    return None

In [81]:
#Usuarios más parecidos dada las distnacias e indices respecto a un usuario
def similar_users(distances, indices):
    list_similarities = []
    for i in range(0,len(distances.flatten())):
        nearest_user = encontrar_id_por_indice(user_ids_dict,indices.flatten()[i])
        
        list_similarities.append((df_prueba[df_prueba['Customer ID'] == nearest_user],distances.flatten()[i]))

    return list_similarities


12347.0


<a id='8'></a><center> <h1 style="background-color:orange; color:white" ><br>Modelo predictivo de recomendaciones basado en KNN<br></h1>

## Recomendación Basada en Vecinos Más Cercanos  

Para un usuario aleatorio, identificamos los **tres vecinos más cercanos** en función de su historial de compras. Esta proximidad se determina utilizando métricas de similitud en el consumo.  

### Proceso de Recomendación  
1. **Identificación de Vecinos Cercanos**  
   - Se seleccionan los tres usuarios más similares en términos de consumo de productos.  

2. **Generación de Recomendaciones**  
   - Se recomiendan al usuario los artículos comprados por sus vecinos que él aún no ha adquirido.  

### Información Adicional  
Este enfoque no solo permite generar recomendaciones personalizadas, sino que también aporta **insights valiosos sobre la segmentación de clientes**. De hecho, este mismo principio es utilizado en el algoritmo **K-Nearest Neighbors (KNN)** cuando se emplea para **clustering**, facilitando la agrupación de clientes según patrones de compra similares.  


In [115]:
#Tomamos al usuario al azar, su indice en la matriz y la lista de compras del mismo
user = np.random.choice(users, size=1)[0]
target_user_index = user_ids_dict[user]
user_purchases = df_prueba[df_prueba['Customer ID'] == user]

#Buscamos vecinos más cercanos al usuario dado
distances, indices = kneighborsForUser(3,user)
print('Usuarios más cercanos: ', list(pd.Series(indices[0]).apply(lambda x: encontrar_id_por_indice(user_ids_dict,x))))
# Items que el usuario objetivo no ha comprado
items_usuario_objetivo = set(np.where(matriz[target_user_index] > 0)[0])  # Items comprados
items_no_comprados = set(range(matriz_normalized.shape[1])) - items_usuario_objetivo  # Items no comprados

# Items comprados por los vecinos
items_vecinos = set()
for vecino_index in indices[0]:
    items_vecinos.update(np.where(matriz[vecino_index] > 0)[0])

# Recomendar items no comprados por el usuario pero sí por los vecinos
recomendaciones = [items[i] for i in items_no_comprados.intersection(items_vecinos)]
print("Items recomendados:", recomendaciones)

Usuarios más cercanos:  [12413.0, 12437.0, 12373.0]
Items recomendados: ['SET 6 FOOTBALL CELEBRATION CANDLES', 'SET OF 20 KIDS COOKIE CUTTERS', 'SET OF 3 BUTTERFLY COOKIE CUTTERS', 'SET OF 3 HEART COOKIE CUTTERS', 'SET OF 36 DINOSAUR PAPER DOILIES', 'DOORMAT FANCY FONT HOME SWEET HOME', 'SET OF 36 PAISLEY FLOWER DOILIES', 'SET OF 9 HEART SHAPED BALLOONS', 'SET/10 BLUE POLKADOT PARTY CANDLES', 'JUMBO STORAGE BAG SUKI', 'GIRLS VINTAGE TIN SEASIDE BUCKET', 'SKULL LUNCH BOX WITH CUTLERY ', 'SPACEBOY BIRTHDAY CARD', 'STRAWBERRY LUNCH BOX WITH CUTLERY', 'PAPER BUNTING RETRO SPOTS', 'RED GINGHAM ROSE JEWELLERY BOX', 'WHITE HEART CONFETTI IN TUBE', '3 PIECE SPACEBOY COOKIE CUTTER SET', 'BAG 125g SWIRLY MARBLES', 'Manual', 'PARTY TIME PENCIL ERASERS', 'MINI JIGSAW CIRCUS PARADE ', 'PARTY CONE CHRISTMAS DECORATION ', 'MINI JIGSAW DINOSAUR ', 'POPCORN HOLDER', 'RED METAL BEACH SPADE ', 'MINI JIGSAW DOLLY GIRL', 'CHERRY BLOSSOM  DECORATIVE FLASK', 'SET/4 BADGES BEETLES', 'MINI JIGSAW SPACEBOY', 'B

# Filtrado y Priorización de Recomendaciones

En esta sección, se abordará el filtrado y ranking de recomendaciones para un usuario específico. Para ello, se asignará una puntuación a los ítems recomendados con el objetivo de evaluar su relevancia.  

## Criterios de Priorización  

Para determinar la importancia de cada recomendación, se consideran los siguientes factores:  

### Frecuencia en los Vecinos Cercanos  

Una forma de priorizar ciertas recomendaciones es analizar la frecuencia con la que los vecinos más cercanos han comprado los ítems recomendados. Aquellos productos que aparecen con mayor frecuencia en los vecinos serán considerados más relevantes para el usuario.  

### Popularidad Global  

Otro criterio relevante es la popularidad de los ítems en la población total. Si un ítem es adquirido con frecuencia por los vecinos del usuario pero no es popular en la población general, es más probable que dicho ítem se ajuste a su patrón de consumo específico en lugar de ser una coincidencia estadística.  

Para potenciar este efecto, se empleará la siguiente métrica de relevancia:  

\[
S(i) = \frac{F_{\text{vecinos}}(i)}{F_{\text{global}}(i)}
\]

donde \( F_{\text{vecinos}}(i) \) representa la frecuencia de compra del ítem \( i \) entre los vecinos cercanos, y \( F_{\text{global}}(i) \) es su frecuencia en la población total.  

### Distancia a los Vecinos  

Finalmente, se considera la distancia de los vecinos en la ponderación de los ítems recomendados. Se otorga mayor peso a los vecinos más cercanos, ya que estos comparten patrones de consumo más similares al usuario objetivo.  

En este proyecto, se optó por un modelo que combina estos tres factores: frecuencia en los vecinos, popularidad global y proximidad de los vecinos, permitiendo generar recomendaciones más precisas y personalizadas.  


In [144]:
from collections import defaultdict

user = np.random.choice(users, size=1)[0]
target_user_index = user_ids_dict[user]
user_purchases = df_prueba[df_prueba['Customer ID'] == user]


distancias, indices = kneighborsForUser(3,user)

# Contar la frecuencia de compra de los items entre los vecinos
frecuencia_items = np.zeros((len(distancias[0]),len(items)))
count_vecino = 0
for vecino_index in indices[0]:
    for item_index in np.where(matriz[vecino_index] > 0)[0]:
        frecuencia_items[count_vecino][item_index] += 1
    count_vecino += 1

# Calcular la popularidad global de cada item
popularidad_global = np.sum(matriz > 0, axis=0)

# Ponderar los items combinando frecuencia, similitud y popularidad global
count_vecino = 0
peso_items = defaultdict(float)
for vecino_index, distancia in zip(indices[0], distancias[0]):
    for item_index in np.where(matriz[vecino_index] > 0)[0]:
        peso_items[item_index] += ((1 - distancia)*frecuencia_items[count_vecino][item_index]/popularidad_global[item_index]) #+ popularidad_global[item_index])/(frecuencia_items[count_vecino][item_index] + popularidad_global[item_index])
    count_vecino += 1

# Ordenar los items por peso (de mayor a menor)
items_ordenados = sorted(peso_items.items(), key=lambda x: x[1], reverse=True)

# Filtrar solo los items no comprados por el usuario objetivo
recomendaciones_priorizadas = [items[item] for item, peso in items_ordenados if item in items_no_comprados]
recomendaciones_priorizadas = list(set(recomendaciones_priorizadas) - set(user_purchases))

print(recomendaciones_priorizadas)

print("\nRecomendaciones:")
for i, item  in enumerate(recomendaciones_priorizadas[0:20], start=1):
    print(f"{i}: {item}")

print("\nCompras del cliente:")
print(user_purchases)

['WHEELBARROW FOR CHILDREN ', 'PINK  POLKADOT PLATE ', 'RED KITCHEN SCALES', 'GREEN METAL BOX ARMY SUPPLIES', 'WOODEN CROQUET GARDEN SET', 'PORCELAIN ROSE SMALL', 'JUMBO BAG VINTAGE DOILY ', 'RED RETROSPOT SHOPPER BAG', 'ASSORTED BOTTLE TOP  MAGNETS ', 'PINK  POLKADOT CUP', 'DOLLY GIRL BABY GIFT SET', 'CIRCUS PARADE LUNCH BOX ', 'PINK POLKADOT BOWL', '20 DOLLY PEGS RETROSPOT', 'CHILDRENS TOY COOKING UTENSIL SET', 'SET OF 2 TRAYS HOME SWEET HOME', 'RED RETROSPOT ROUND CAKE TINS', 'DOORSTOP RETROSPOT HEART', 'BIRTHDAY PARTY CORDON BARRIER TAPE', 'RETROSPOT LAMP', 'BOX OF VINTAGE JIGSAW BLOCKS ', 'NATURAL SLATE CHALKBOARD LARGE ', 'VICTORIAN GLASS HANGING T-LIGHT', 'SET OF 4 KNICK KNACK TINS DOILY ', 'CIRCUS PARADE BABY GIFT SET', 'PINK BABY BUNTING', 'FIVE CATS HANGING DECORATION', 'SET OF 72 PINK HEART PAPER DOILIES', 'PICNIC BOXES SET OF 3 RETROSPOT ', 'RED RETROSPOT CUP', 'LUNCH BAG PINK POLKADOT', 'BLUE POLKADOT PLATE ', 'CHILDS BREAKFAST SET DOLLY GIRL ', 'SPACEBOY LUNCH BOX ', 'REC

<a id='8'></a><center> <h1 style="background-color:orange; color:white" ><br>Analisis de las predicciones<br></h1>



## Patrones en las Compras del Cliente  
El cliente ha comprado en grandes cantidades productos relacionados con:  

### Juguetes y artículos infantiles  
- 4 Traditional Spinning Tops  
- Set of 6 Soldier Skittles  
- Wooden Box of Dominoes  
- Spaceboy Mini Backpack  
- Spaceboy Wall Art  
- Children's Toy Cooking Utensil Set  
- Rabbit Design Cotton Tote Bag  

### Artículos de cocina y repostería  
- Baking Set 9 Piece Retrospot  
- Jam Making Set with Jars  
- Set of 20 Kids Cookie Cutters  
- Red Kitchen Scales  
- Ivory Kitchen Scales  
- Retrospot Tea Set Ceramic 11 PC  

### Accesorios decorativos  
- Cream Sweetheart Mini Chest  
- Classic Glass Cookie Jar  
- Classic Metal Birdcage Plant Holder  
- Doorstop Retrospot Heart  
- Regency Cakestand 3 Tier  
- Set of 3 Regency Cake Tins  

### Bolsas y almacenamiento  
- Jumbo Bag Red Retrospot  
- Jumbo Bag Vintage Doily  
- Lunch Bag Pink Polkadot  
- Lunch Bag Red Retrospot  
- Lunch Bag Vintage Doily  

## Recomendaciones  
Tomando el **top 20** de ítems recomendados por el algoritmo, encontramos bastantes coincidencias con el patrón de compra del usuario:  

- **Productos infantiles y juguetes:**  
  - Wheelbarrow for Children  
  - Wooden Croquet Garden Set  
  - Dolly Girl Baby Gift Set  

- **Productos de cocina y repostería:**  
  - Red Kitchen Scales  
  - Pink Polkadot Plate  
  - Pink Polkadot Bowl  

- **Accesorios decorativos:**  
  - Doorstop Retrospot Heart  
  - Set of 2 Trays Home Sweet Home  

- **Bolsas y almacenamiento:**  
  - Jumbo Bag Vintage Doily  
  - Red Retrospot Shopper Bag  

En general, las recomendaciones son bastante acertadas, alineándose bien con los hábitos de compra del cliente. Sin embargo, se podría mejorar afinando aún más la selección para priorizar artículos de cocina infantil y juguetes tradicionales.
